<a href="https://colab.research.google.com/github/CarlosFinatti/Projeto_Filmes_PLN_2025.3/blob/main/Projeto_Filmes_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!pip install langchain
!pip install langchain-google-genai
!pip install langchain_text_splitters
!pip install nltk
!pip install langchain-classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00


In [12]:
from IPython.utils.text import indent
import os
import spacy
import requests
import nltk
from datetime import datetime as dt
from langchain_classic.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_classic.chains.summarize import load_summarize_chain
from google.colab import userdata
from google.colab import auth
from collections import Counter
from nltk.corpus import stopwords

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
TMDB_API_KEY = userdata.get('TMDB_API_KEY')
sinopse_teste = userdata.get('sinopse_teste')
auth.authenticate_user()

#a llm vai traduzir pro inglês já
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')

def extrai_keywords(text, n_keywords=10):
    """funçao pra extrair as palavras chaves"""
    doc = nlp(text)

    keywords = [token.lemma_.lower() for token in doc if token.pos_ in ['NOUN', 'ADJ'] and token.lemma_.lower() not in stopwords.words('english')]
    freq = Counter(keywords)
    return [word for word, _ in freq.most_common(n_keywords)]

def sumariza_sinopse(text, llm):
    """quebra pedaços do texto e resume cada um, depois junta tudo e monta o resumo"""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    docs = [Document(page_content=chunk) for chunk in text_splitter.split_text(text)]
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    resumo = chain.run(docs)
    return resumo

def prepara_sinopse_pro_llm(sinopse, llm):
    """prepara a sinopse tratada: extrai keywords, sumariza e combina."""
    keywords = extrai_keywords(sinopse)
    sinopse_resumida = sumariza_sinopse(sinopse, llm)

    sinopse_tratada = f"Resumo: {sinopse_resumida}\nPalavras-chave: {', '.join(keywords)}"
    return sinopse_tratada

def determina_generos(sinopse_tratada, llm):
    """tira os generos da sinopse ja tratada"""
    prompt = PromptTemplate(
        input_variables=["synopsis"],
        template="Com base na sinopse tratada abaixo, identifique os prováveis gêneros do filme (ex.: ação, drama, ficção científica). Liste-os em uma lista separada por vírgulas.\n\n{synopsis}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    generos = chain.invoke(input=sinopse_tratada)
    return generos

def transcreve_generos(generos):
  """pega os generos que o llm sugeriu, transcreve de acordo com os ids do TMDB e monta uma string para por na url de procura"""
  lista_generos = generos.split(', ')
  ind = lista_generos.index('Suspense')
  lista_generos[ind] = 'Thriller'
  generos_brutos = []
  id_generos = []
  url = f'https://api.themoviedb.org/3/genre/movie/list?api_key={TMDB_API_KEY}&language=pt-BR'
  r = requests.get(url).json()
  generos_brutos.extend(r['genres'])
  for d in generos_brutos:
    if d['name'] in lista_generos:
      id_generos.append(d['id'])
  id_generos_str = str(id_generos)[1:-1].replace(', ', ',')
  return id_generos_str

def procura_filmes_similares(generos):
  """lista os filmes dos ultimos 5 anos baseado nos generos e depois expoe os titulos e as sinopses dos filmes encontrados"""
  ano_atual = dt.now().year
  ano_final = ano_atual-5
  pagina = 1
  filmes = []

  while True:
    url = f'https://api.themoviedb.org/3/discover/movie?api_key={TMDB_API_KEY}&primary_release_year.gte={ano_final}-01-01&primary_release_year.lte={ano_atual}-12-31&with_genres={transcreve_generos(generos)}&page={pagina}'
    r = requests.get(url).json()
    filmes.extend(r['results'])
    if pagina > 5:
      break
    pagina += 1

  for exp_filme in filmes:
    print(f'Título: {exp_filme['title']}\n{'----'*20}')
  return

def main():
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", google_api_key=GEMINI_API_KEY, temperature=0.1)

    #sinopse = input("Digite a sinopse do filme: ")
    print(sinopse_teste+'\n')
    sinopse = sinopse_teste

    sinopse_tratada = prepara_sinopse_pro_llm(sinopse, llm)
    generos = determina_generos(sinopse_tratada, llm)
    generos_obtidos = generos.get('text')
    print(f"{sinopse_tratada}\n")
    print(f"\nGêneros prováveis: {generos_obtidos}")
    print(transcreve_generos(generos_obtidos))
    procura_filmes_similares(generos_obtidos)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


A group of serial killers meet in an abandoned storage facility, trying to find out what happened with their lost memories. Now, they must find a way to retrieve their memories, find out what is and how to stop the bloody ritual known as the Hexatombe.

Resumo: Amnesiac serial killers must recover their memories to prevent a deadly ritual called the Hexatombe.
Palavras-chave: memory, group, serial, killer, storage, facility, lost, way, bloody, ritual


Gêneros prováveis: Suspense, Terror, Mistério
27,9648,53
Título: The Dogs
--------------------------------------------------------------------------------
Título: Bone Lake
--------------------------------------------------------------------------------
Título: Psycho III
--------------------------------------------------------------------------------
Título: From Darkness
--------------------------------------------------------------------------------
Título: The Mannequin
----------------------------------------------------------------